In [1]:
import glob
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn import preprocessing

In [2]:
df = pd.concat([pd.read_parquet(path=path) for path in glob.glob('/Users/choisj2/data/da-datalake-prod/ml-mart/rds/product/year=2021/month=06/day=15/*')[:20]])

KeyboardInterrupt: 

In [ ]:
# df.info()

In [ ]:
dff = df[['C_PRDUCT_NO', 'C_PRDUCT_DSCNT_PC', 'C_PRDUCT_SELL_PC', 'TAG_NO', 'C_PRDUCT_KND_CD', 'PROD_STYLE_NM','PROD_COLOR_NM', 'C_MD_NO', 'C_CTGRY_NO_THIRD']].copy()
dff = dff.dropna()
dff = dff.astype({'C_CTGRY_NO_THIRD':'int64'})

In [ ]:
# dff['TAG_NO'] = dff['TAG_NO'].str.split(',')

In [ ]:
dff.replace({"C_PRDUCT_KND_CD": {"B":0, "N":1, "S":2}}, inplace=True)
dff.replace({"PROD_STYLE_NM": {"선택안함":0, "심플베이직":1, "캐주얼":2, "오피스룩":3, "페미닌":4, "바캉스": 5, "러블리":6, "섹시글램": 7, "스포츠":8}}, inplace=True)
dff.replace({"PROD_COLOR_NM": {"선택안함":0, "블랙":1, "그레이":2, "화이트":3, "네이비":4, "베이지":5, "블루":6, "와인":7, "레드":8, "핑크":9, "스카이":10, "라벤더":11, "옐로우":12, "민트":13,"실버":14,"그린":15,"카키":16,"골드":17,"퍼플":18,"로즈골드":19,"오렌지":20, "브라운": 21}}, inplace=True)

In [ ]:
# dff = dff.join(pd.get_dummies(dff['TAG_NO'].apply(pd.Series).stack()).sum(level=0))

In [ ]:
# dff = dff.fillna(0)
dff.drop('TAG_NO', axis = 1, inplace=True)
#dff_x = dff.drop('C_CTGRY_NO_THIRD', axis = 1)

In [ ]:
features = dff_x[['C_PRDUCT_NO', 'C_PRDUCT_DSCNT_PC', 'C_PRDUCT_SELL_PC', 'TAG_NO', 'C_PRDUCT_KND_CD', 'PROD_STYLE_NM','PROD_COLOR_NM', 'C_MD_NO']].values
label = dff['C_CTGRY_NO_THIRD'].values
kfold = KFold(n_splits = 5)
dt_clf = DecisionTreeClassifier(random_state=156)

In [ ]:
cv_accuracy = []
n_iter = 0
for train_index, test_index in kfold.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    cv_accuracy.append(accuracy)
    print('{0} 교차 검증 정확도 : {1}, 학습 데이터의 크기 : {2}, 검증 데이터 크기 : {3}'.format(n_iter, accuracy, X_train.shape[0], X_test.shape[0]))